# LangGraph Multi-Agent System

This notebook implements the LangGraph Multi-Agent System using Groq's LLM API. All the necessary code is included directly in this notebook.

## 1. Install Required Packages

In [ ]:
!pip install langchain>=0.1.0 langchain-community>=0.0.10 langgraph>=0.0.10 pydantic>=2.0.0 python-dotenv>=1.0.0 tavily-python>=0.1.9 requests>=2.31.0 groq>=0.4.0 langchain-core>=0.1.0

## 2. Set Up Environment Variables

In [ ]:
from google.colab import userdata
import os

# Set up environment variables
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')
os.environ['EMAIL_ADDRESS'] = userdata.get('EMAIL_ADDRESS')
os.environ['EMAIL_CODE'] = userdata.get('EMAIL_CODE')

## 3. Define the Plan Agent

In [ ]:
from typing import List, Dict, Any
from pydantic import BaseModel
from langchain_core.messages import HumanMessage
from groq import Groq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

class SubTask(BaseModel):
    task_id: str
    description: str
    status: str = "pending"
    result: str = ""
    feedback: str = ""

class PlanAgent:
    def __init__(self, client: Groq):
        self.client = client
        self.system_prompt = """You are a planning agent responsible for breaking down complex tasks into smaller, manageable sub-tasks.
        For each user request, you should:
        1. Analyze the request and identify the main components
        2. Break it down into logical sub-tasks
        3. Ensure sub-tasks are clear, specific, and actionable
        4. Consider dependencies between tasks
        5. Return a list of sub-tasks in order of execution"""

    def plan(self, state: Dict[str, Any]) -> List[SubTask]:
        """Generate a plan by breaking down the user's request into sub-tasks."""
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": state["messages"][-1].content}
        ]
        
        response = self.client.chat.completions.create(
            messages=messages,
            model="llama-3.3-70b-versatile",
            temperature=0.7
        )
        
        result = response.choices[0].message.content
        
        # Parse the LLM response into SubTask objects
        tasks = []
        for i, task_desc in enumerate(result.split('\n')):
            if task_desc.strip():
                tasks.append(SubTask(
                    task_id=f"task_{i+1}",
                    description=task_desc.strip(),
                    status="pending"
                ))
        return tasks

## 4. Define the Tool Agent

In [ ]:
from typing import Dict, Any, List
from pydantic import BaseModel
from langchain_core.messages import HumanMessage
from groq import Groq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.prebuilt import create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from tavily import TavilyClient
import smtplib
from email.mime.text import MIMEText

class ToolAgent:
    def __init__(self, client: Groq):
        self.client = client
        self.tools = {
            'search': TavilySearchResults(max_results=5),
            'extract': TavilyClient(api_key=os.getenv('TAVILY_API_KEY')).extract,
            'email': self._send_email,
            'generate_image': self._generate_image,
            'generate_audio': self._generate_audio
        }
        
        self.system_prompt = """You are a tool agent responsible for executing specific tasks using available tools.
        For each task, you should:
        1. Identify the appropriate tool for the task
        2. Execute the task using the selected tool
        3. Return the result and any relevant feedback
        4. Handle errors gracefully and provide meaningful feedback"""

    def execute_task(self, state: Dict[str, Any], task: Dict[str, Any]) -> Dict[str, Any]:
        """Execute a specific task using the appropriate tool."""
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": task["description"]}
        ]
        
        response = self.client.chat.completions.create(
            messages=messages,
            model="llama-3.3-70b-versatile",
            temperature=0.7
        )
        
        result = response.choices[0].message.content
        
        # Execute the appropriate tool based on the task
        tool_name = self._identify_tool(result)
        if tool_name in self.tools:
            tool_result = self.tools[tool_name](task["description"])
            feedback = self._generate_feedback(tool_result)
            return {
                "result": tool_result,
                "status": "completed",
                "feedback": feedback
            }
        else:
            return {
                "result": "No appropriate tool found for this task",
                "status": "failed",
                "feedback": "Please try rephrasing the task"
            }

    def _send_email(self, subject: str, body: str, to_email: str) -> None:
        """Send an email using SMTP."""
        smtp_server = 'smtp.gmail.com'
        smtp_port = 587
        sender_email = os.getenv('EMAIL_ADDRESS')
        password = os.getenv('EMAIL_CODE')

        message = MIMEText(body, 'plain')
        message['From'] = sender_email
        message['To'] = to_email
        message['Subject'] = subject

        try:
            server = smtplib.SMTP(smtp_server, smtp_port)
            server.starttls()
            server.login(sender_email, password)
            server.sendmail(sender_email, to_email, message.as_string())
            server.quit()
            print("Email sent successfully!")
        except Exception as e:
            print(f"Failed to send email: {e}")

    def _generate_image(self, prompt: str) -> str:
        """Generate an image using Groq."""
        try:
            response = self.client.chat.completions.create(
                messages=[{"role": "user", "content": f"Generate an image based on this description: {prompt}"}],
                model="llama-3.3-70b-versatile",
                temperature=0.7
            )
            return f"Image generated successfully: {response.choices[0].message.content}"
        except Exception as e:
            print(f"Error generating image: {e}")
            return None

    def _generate_audio(self, text: str, voice: str = "alloy") -> str:
        """Generate audio using Groq."""
        try:
            response = self.client.chat.completions.create(
                messages=[{"role": "user", "content": f"Generate audio for this text: {text}"}],
                model="llama-3.3-70b-versatile",
                temperature=0.7
            )
            return f"Audio generated successfully: {response.choices[0].message.content}"
        except Exception as e:
            print(f"Error generating audio: {e}")
            return None

## 5. Define the Workflow

In [ ]:
from typing import Dict, Any, List, TypedDict, Annotated
from langchain_core.messages import BaseMessage
from langgraph.graph import END, START, StateGraph
import operator
from groq import Groq
from langchain_core.messages import HumanMessage
import os

class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    tasks: List[SubTask]
    current_task_index: int
    feedback: str

def create_workflow():
    # Initialize agents with Groq
    client = Groq(api_key=os.getenv("GROQ_API_KEY"))
    plan_agent = PlanAgent(client)
    tool_agent = ToolAgent(client)
    
    # Create workflow graph
    workflow = StateGraph(AgentState)
    
    # Define nodes
    def plan_node(state: Dict[str, Any]) -> Dict[str, Any]:
        """Generate initial plan or update plan based on feedback."""
        if not state.get("tasks"):
            # Initial planning
            tasks = plan_agent.plan(state)
        else:
            # Update plan based on feedback
            tasks = plan_agent.update_plan(state, state.get("feedback", ""))
        
        return {
            "messages": state["messages"],
            "tasks": tasks,
            "current_task_index": 0,
            "feedback": ""
        }
    
    def execute_node(state: Dict[str, Any]) -> Dict[str, Any]:
        """Execute the current task."""
        current_task = state["tasks"][state["current_task_index"]]
        result = tool_agent.execute_task(state, current_task.dict())
        
        # Update task with result and feedback
        current_task.result = result["result"]
        current_task.status = result["status"]
        current_task.feedback = result["feedback"]
        
        return {
            "messages": state["messages"] + [HumanMessage(content=result["result"])],
            "tasks": state["tasks"],
            "current_task_index": state["current_task_index"],
            "feedback": result["feedback"]
        }
    
    def should_continue(state: Dict[str, Any]) -> str:
        """Determine if we should continue with the next task or go back to planning."""
        if state["current_task_index"] >= len(state["tasks"]) - 1:
            return "plan" if state["feedback"] else "end"
        return "execute"
    
    # Add nodes to workflow
    workflow.add_node("plan", plan_node)
    workflow.add_node("execute", execute_node)
    
    # Add edges
    workflow.add_edge(START, "plan")
    workflow.add_conditional_edges(
        "execute",
        should_continue,
        {
            "plan": "plan",
            "execute": "execute",
            "end": END
        }
    )
    
    # Compile workflow
    return workflow.compile()

## 6. Test the Environment

In [ ]:
def test_environment():
    # Test Groq API
    try:
        client = Groq(api_key=os.getenv('GROQ_API_KEY'))
        response = client.chat.completions.create(
            messages=[{"role": "user", "content": "Hello"}],
            model="llama-3.3-70b-versatile"
        )
        print("✅ Groq API key is working")
    except Exception as e:
        print("❌ Groq API key error:", str(e))
    
    # Test Tavily
    try:
        tavily_client = TavilyClient(api_key=os.getenv('TAVILY_API_KEY'))
        response = tavily_client.search("test query")
        print("✅ Tavily API key is working")
    except Exception as e:
        print("❌ Tavily API key error:", str(e))
    
    # Test Email
    email_code = os.getenv('EMAIL_CODE')
    if email_code and len(email_code.replace(" ", "")) == 16:
        print("✅ Email configuration looks correct")
    else:
        print("❌ Email configuration error: Make sure you have a valid 16-character app password")

test_environment()

## 7. Run the Workflow

In [ ]:
def run_workflow(task_description: str):
    workflow = create_workflow()
    
    print("\n🤖 Processing your request...\n")
    
    # Run workflow with initial user query
    for state in workflow.stream({
        "messages": [HumanMessage(content=task_description)]
    }):
        if "__end__" not in state:
            # Format the output in a more readable way
            if "tasks" in state["plan"]:
                print("📋 Plan Generated:\n")
                for task in state["plan"]["tasks"]:
                    if task.description.startswith("**"):
                        print(f"\n{task.description.strip('*')}")
                    else:
                        print(f"- {task.description}")
                    if task.result:
                        print(f"  Result: {task.result}")
                    if task.feedback:
                        print(f"  Feedback: {task.feedback}")
                print("\n---")

# Example usage
run_workflow("Plan a weekend trip to the mountains. Assign specific responsibilities to each agent.")

## 8. Try Different Prompts

In [ ]:
# Example 1: Research AI frameworks
run_workflow("Research and summarize the top 5 AI frameworks for building chatbots. Each agent should handle one part of the process.")

In [ ]:
# Example 2: Climate change report
run_workflow("You are a team of AI agents working on a report about climate change. One agent should research, one should write, and one should proofread. Coordinate and deliver the final report.")

In [ ]:
# Example 3: To-do list app
run_workflow("Build a basic to-do list web app using React. Assign the coding, documentation, and testing tasks to the appropriate agents.")